In [1]:
import json, requests, os
import logging
from shapely.geometry import shape, MultiLineString, LineString, Point
from rtree import index
from io import BytesIO
import geopandas as gpd

In [2]:
# Load the existing data
# URL of the GeoJSON file
url = 'https://github.com/newzealandpaul/Shipping-Lanes/blob/main/data/Shipping_Lanes_v1.geojson?raw=true'

# Fetch the GeoJSON file
response = requests.get(url)
response.raise_for_status()  # Ensure the request was successful

data = json.loads(response.content)

# Convert features to Shapely LineString objects
major = []
middle = []
minor = []

for feature in data['features']:
    route_type = feature['properties']['Type']
    routes = [] 
    geom = shape(feature['geometry'])
    if isinstance(geom, MultiLineString):
        for line in geom.geoms:
            routes.append(line)
    
    if route_type == 'Major':
        major = routes
    elif route_type == 'Middle':
        middle = routes
    elif route_type == 'Minor':
        minor = routes

# Build spatial indices
def build_spatial_index(routes):
    idx = index.Index()
    for i, line in enumerate(routes):
        idx.insert(i, line.bounds, obj=line)
    return idx

major_idx = build_spatial_index(major)

In [22]:
# Fixed version of find_candidate_route
def find_candidate_route_fixed(idx, routes, lon, lat, distance_threshold=0.0001, num_candidates=10):
    """
    Find the closest route to a given point.
    
    Parameters:
    - idx: Spatial index for route lookup
    - routes: List of route geometries
    - lon, lat: Coordinates of the query point
    - distance_threshold: Maximum distance (in nautical miles) to consider a route
    - num_candidates: Number of nearest candidates to check
    
    Returns:
    - route_id: Index of the route (1-based)
    - route: The route geometry
    - distance_nm: Distance to the route in nautical miles
    """
    # Convert distance threshold from nautical miles to degrees
    # 1 nautical mile = 1/60th of a degree of latitude
    degree_threshold = distance_threshold / 60
    
    query_point = Point(lon, lat)
    
    # Find the nearest candidates
    candidate_indices = list(idx.nearest((lon, lat, lon, lat), num_candidates))
    
    closest_route = None
    closest_id = None
    min_distance = float('inf')
    closest_proj_distance = None
    
    # Check all candidates and find the closest one
    for i in candidate_indices:
        route = routes[i]
        # Find the closest point on the route
        proj_distance = route.project(query_point)
        nearest_point = route.interpolate(proj_distance)
        
        # Calculate distance in degrees
        distance_in_degrees = query_point.distance(nearest_point)
        
        # Convert to nautical miles (1 degree ≈ 60 nautical miles)
        distance_in_nm = distance_in_degrees * 60
        
        # Keep track of the closest route
        if distance_in_nm < min_distance:
            min_distance = distance_in_nm
            closest_route = route
            closest_id = i
            closest_proj_distance = proj_distance
    
    # If the closest route is within threshold, return it
    if min_distance <= distance_threshold:
        return closest_id + 1, closest_route, closest_proj_distance
    
    return None, None, None

In [20]:
# Test with the problematic coordinates
print("Testing with fixed function:")
#i, route, proj_distance = find_candidate_route_fixed(major_idx, major, -125.4321268, 42.8346738, distance_threshold=50)
#i, route, proj_distance = find_candidate_route_fixed(major_idx, major, -129.6383054, 36.1078860, distance_threshold=125)
i, route, proj_distance = find_candidate_route_fixed(major_idx, major, 47.3832503, -7.2682782, distance_threshold=50)

if i is not None:
    # IMPORTANT: proj_distance is NOT the distance to the route - it's the position along the route!
    # We need to calculate the actual minimum distance
    query_point = Point(-129.6383054, 36.1078860)
    nearest_point = route.interpolate(proj_distance)
    
    # Calculate distance in degrees
    distance_in_degrees = query_point.distance(nearest_point)
    
    # Convert to nautical miles (1 degree ≈ 60 nautical miles)
    distance_nm = distance_in_degrees * 60
    
    print(f"Found route {i} with distance {distance_nm:.2f} nautical miles")
    print(f"Position along route: {proj_distance:.2f} (this is not the distance to the route)")
    
    # Print additional debugging info
    print(f"All major routes: {len(major)}")
    print(f"Route coordinates sample: {list(major[i-1].coords)[:3]}")
else:
    print("No route found within the distance threshold.")

Testing with fixed function:
No route found within the distance threshold.


In [21]:
# Additional debugging - check all routes and distances
print("Checking all 5 closest routes:")
query_point = Point(47.3832503, -7.2682782,)
candidate_indices = list(major_idx.nearest((query_point.x, query_point.y, query_point.x, query_point.y), 5))

for i in candidate_indices:
    route = major[i]
    proj_distance = route.project(query_point)
    nearest_point = route.interpolate(proj_distance)
    
    # Calculate distance in degrees
    distance_in_degrees = query_point.distance(nearest_point)
    
    # Convert to nautical miles
    distance_in_nm = distance_in_degrees * 60
    
    print(f"Route {i+1}: Distance = {distance_in_nm:.2f} nm, Proj distance = {proj_distance:.6f}")

Checking all 5 closest routes:
Route 35: Distance = 1180.61 nm, Proj distance = 31.317203
Route 39: Distance = 1989.75 nm, Proj distance = 27.267280
Route 40: Distance = 1200.38 nm, Proj distance = 0.086532
Route 1: Distance = 4206.97 nm, Proj distance = 94.293017
Route 44: Distance = 2383.07 nm, Proj distance = 57.613353
